In [116]:
%matplotlib inline
import math
import sys, os
import spiceypy as spice 
from skimage import data, color, io
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.filters import roberts, sobel, scharr, prewitt
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte
import osgeo.gdal as gdal
import osgeo.gdal_array as gdal_array 
gdal.UseExceptions()
import pandas as pd
import plio
from plio.io.io_gdal import GeoDataset
import pvl

In [169]:
class JunoCam:
    cx = 814.21
    cy = [315.48, 158.48, 3.48, -151.52]
    k1 = -5.9624209455667325e-08
    k2 = 2.7381910042256151e-14
    fl = 1480.5907441315994
    kerns_loaded = False
    
    #def __init__
    
    def undistort(c):
        """Project pixels from the FPA through the JunoCam optics"""
        xd, yd = c[0], c[1]
        for i in range(5):
            r2 = (xd**2+yd**2)
            dr = 1+k1*r2+k2*r2*r2
            xd = c[0]/dr
            yd = c[1]/dr
        return [xd, yd]

    
    def distort(c):
        """Project surface points into through the JunoCam optics to the FPA"""
        xd, yd = c[0], c[1]
        r2 = (xd**2+yd**2)
        dr = 1+k1*r2+k2*r2**2
        xd *= dr
        yd *= dr
        return [xd, yd]

    
    def vector2xy(v, band):
        """Find the pixel location based on a light ray from the target"""
        alpha = v[2]/fl
        cam = [v[0]/alpha, v[1]/alpha]
        cam = distort(cam)
        x = cam[0]+cx
        y = cam[1]+cy[band]
        return (x, y)

    
    def xy2vector(x, y, band):
        """Project a pixel to a look vector"""
        cam = [x-cx, y-cy[band]]
        cam = undistort(cam)
        v = (cam[0], cam[1], fl)
        return v

    def getkerns(self):
        kerns_loaded = True
        """Load the SPICE kernels."""
        spice.furnsh("naif0012.tls")
        spice.furnsh("pck00010.tpc")
        spice.furnsh(os.getenv("NAIFSPK"))
        spice.furnsh(os.getenv("NAIFCK"))
        spice.furnsh("juno_v12.tf")
        spice.furnsh("jup310.bsp")
        spice.furnsh("JNO_SCLKSCET.00072.tsc")
        
    def mark(x, y, color):
        """Mark a pixel. Usually used for finding a pixel that overlaps the target."""
        draw.point((x, y), fill=color)

    def scan(t, f_offset, band, color):
        """scan over the framelet with the given offsets and mark all pixels that hit the planet"""
        for y in range(0, 128, 4):
            for x in range(0, 1600, 4):
                # construct the vector in juno_junocam space...
                v = xy2vector(x, y, band)
                # and see if it intersects the target.
                vx = spice.sincpt("Ellipsoid", "jupiter", t, \
                                  "iau_jupiter", "lt+s", "juno", "juno_junocam", v)
                if vx:
                    mark(x, y+f_offset, color)

       
    def planet(t, f_offset, band, color):
        """plot the planet's nominal limb"""
        to_targ = spice.spkezr(targ, t, "iau_jupiter", "LT+S", "juno")[0][0:3]
        e = spice.edlimb(radii[0], radii[1], radii[2], vminus(to_targ))
        c = spice.pxform("iau_jupiter", "juno_junocam", t)
        org = spice.vadd(e.center, to_targ)
        for th in range(-1800, 1800):
            p = spice.vadd(org, vadd(vscl(math.cos(math.radians(th/10.0)), e.semi_major), \
                                     vscl(math.sin(math.radians(th/10.0)), e.semi_minor)))
            x, y = junocamlib.vector2xy(mxv(c, p), band)
            if x >= 0 and x < 1648 and y >= 0 and y < 128:
                mark(x, y+f_offset, color)
  
    def find(f_offset, band, color):
        for dt in range(0, nframes, 1):
            t = t0+dt*(interframe)
            #scan(t, f_offset+128*3*dt, band, color)
            planet(t, f_offset+128*3*dt, band, color)

    
    def readimage(self, input_file):
        """Read in the PDS image"""
        imageds = gdal.Open(input_file, gdal.GA_ReadOnly)
        metadata =pvl.load(input_file)
        #print(metadata)
        start_time = metadata["SPACECRAFT_CLOCK_START_COUNT"]
        interframe = metadata["INTERFRAME_DELAY"]
        imrb = imageds.GetRasterBand(1)
        im = imrb.ReadAsArray()
        print(im.size)
        nframes = int(im.size/384/1648)
        print(nframes)# only works for RGB images
        t0 = spice.scs2e(-61, start_time)+4094/50e3-20e-3 # metadata start time is 61.88 milliseconds early
        interframe = float(im.info["interframe"])+1e-3 # metadata interframe is 1 millisecond short
        print(t0, spice.et2utc(t0, "isoc", 3), interframe)
        targ = "jupiter"
        radii = spice.bodvrd(targ, "RADII", 3)[1]
        
        
    def draw():
        find(0, 1, "blue")
        find(128, 2, "green")
        find(256, 3, "red")
        im.save("map%s.ddd"%sys.argv[1][:-4])

In [170]:
jc = JunoCam()

In [171]:
jc.getkerns()
jc.readimage("JNCE_2017038_04C00586_V01.LBL")

AttributeError: 'NoneType' object has no attribute 'encode'